### 머신러닝을 위한 타이타닉 데이터처리

In [ ]:
# 구글 Colab에서 공유된 파일 다운로드해서 가져오기 - 타이타닉데이터
#!gdown https://drive.google.com/uc?id=1EKULCfCFtu3ThmTUMlpUDV1nuJIbjcHP

In [13]:
import pandas as pd
import numpy as np

# 1. 판다스 read_csv()를 사용하여'titanic.csv' 데이터를 읽어옵니다.

df = pd.read_csv("./data/titanic.csv")          # ... 코드 입력
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
# 읽어온 데이터 정보파악하기
df.info()          # ... 코드 입력

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## 1. 텍스트 데이터 숫자 데이터로 변환하기

### 1) replace() 사용하여 데이터 값을 바꾸기

In [15]:
# 'Sex' 컬럼 상위 5개 불러오고 컬럼 카테고리 파악하기
print('변환 전: \n',df['Sex'].head())
df['Sex'].value_counts()

변환 전: 
 0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object


male      577
female    314
Name: Sex, dtype: int64

In [17]:
# replace()를 사용하여 male -> 0, female -> 1로 변환합니다.
# 정석은 
#df["Sex"] = df["Sex"].replace({"male":0, "female":1})          # ... 코드 입력
df = df.replace({"male":0, "female":1})          # ... 코드 입력

# 변환한 성별 데이터를 출력합니다.
print('\n변환 후: \n',df['Sex'].head())


변환 후: 
 0    0
1    1
2    1
3    1
4    0
Name: Sex, dtype: int64


### 2) get_dummies() 사용하여 원핫인코딩하기

In [18]:
# 'Embarked' 컬럼 상위 5개 불러오고 컬럼 카테고리 파악하기
print('변환 전: \n',df['Embarked'].head())
df['Embarked'].value_counts()

변환 전: 
 0    S
1    C
2    S
3    S
4    S
Name: Embarked, dtype: object


S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [21]:
# get_dummies()를 사용하여  'Embarked' 컬럼 변환합니다.
dummies = pd.get_dummies(df[["Embarked"]])          # ... 코드 입력

#[[]]

# 변환한 Embarked 데이터를 출력합니다.
print('\n변환 후: \n',dummies.head())


변환 후: 
    Embarked_C  Embarked_Q  Embarked_S
0           0           0           1
1           1           0           0
2           0           0           1
3           0           0           1
4           0           0           1


## 2. 숫자 데이터를 정규화, 표준화시키기
- 데이터의 컬럼 별 단위 또는 범위를 통일시켜주기 위해서임 (컬럼별로 시간, km, kg 등 다양한 단위를 가지는데 이러면 직접적인 비교가 불가능함)  

- 머신러닝에 사용되는 데이터는 그 값이 너무 크거나 작지 않고 적당한 범위 (-1에서 ~ 1사이)에 있어야 모델의 정확도가 높아진다고 알려져있음

- 머신러닝에서 정규화 또는 표준화 중 무엇을 해야하는지는 2개를 비교해보고 결정해야함. 어느 모델에선 정규화를 한 데이터가 값이 더 높을 수도 있고 표준화를 한 데이터가 값이 더 높을 수가 있기 때문에  둘 다 비교해 보는 것을 추천함

#### 정규화(Normalization)

- 데이터를 특정 구간으로 바꾸는 척도법이다. 즉 값의 범위를 0~1사이로 옮겨준다.  (ex. 0~1 or 0~100).
- 식 : (측정값 - 최소값) / (최대값 - 최소값)
- 모든 데이터의 단위가 모두 0~1사이로 옮겨지게 해주는 것이다.
- 이러면 데이터의 컬럼들이 평등하게 0~1사이로 놓여지기 때문에 기계가 학습하기에 어느 컬럼에 중점을 두고 학습하기보단 평등하게 컬럼들을 보고 학습시킨다.

In [22]:
# 1) 정규화를 수행하는 함수를 구현합니다.
def normal(data):
    data =(data- data.min()) / (data.max()- data.min())
    return data

print('변환 전: \n',df['Fare'].head())

# normal 함수를 사용하여 정규화합니다.
Fare = normal(df['Fare'])

# 변환한 Fare 데이터를 출력합니다.
print('\n변환 후: \n',Fare.head())

변환 전: 
 0     7.2500
1    71.2833
2     7.9250
3    53.1000
4     8.0500
Name: Fare, dtype: float64

변환 후: 
 0    0.014151
1    0.139136
2    0.015469
3    0.103644
4    0.015713
Name: Fare, dtype: float64


#### 표준화(Standardization)

- 데이터를 0을 중심으로 양쪽으로 데이터를 분포시키는 방법이다. 표준화를 하게 되면 각 데이터들은 평균을 기준으로 얼마나 떨여져 있는지를 나타내는 값으로 변환된다.
- 식 (Z-score 표준화) : (측정값 - 평균) / 표준편차
- 변환된 데이터는 다소 평평하게 만드는 특성을 가진다 (진폭의 감소). 진폭의 감소로 각 데이터의 간격이 감소하게 된다 (ex. 10000의 단위에서 0.1 단위로 감소).

- 표준화는 먼저 데이터가 정규분포를 따른다는 가정하에 실시된다.(정규분포는 종모양 분포임)

- 정규분포를 따른다는 가정하에 데이터를 평균은 0, 표준편차는 1이 되도록 만들어준다. 그렇기에 변한 값이 0~1사이로 바뀌지는 않는다. 하지만 평균은 0이니깐 0의 근처로 많이 잡힐 것이다.

In [23]:
# 1) 표준화를 수행하는 함수를 구현합니다.
def standardize(data):
    data = (data - data.mean()) / data.std()
    return data

print('변환 전: \n', df['Fare'].head())

# standardize 함수를 사용하여 표준화합니다.
Fare_standardized = standardize(df['Fare'])

# 변환한 Fare_standardized 데이터를 출력합니다.
print('\n변환 후: \n', Fare_standardized.head())


변환 전: 
 0     7.2500
1    71.2833
2     7.9250
3    53.1000
4     8.0500
Name: Fare, dtype: float64

변환 후: 
 0   -0.502163
1    0.786404
2   -0.488580
3    0.420494
4   -0.486064
Name: Fare, dtype: float64


## 3. 데이터 정제 및 분리하기 - 결측값&이상치 처리하기

### 1) 결측치 처리하기

In [25]:
# 결측치 파악하여 갯수 구해보기 - isnull(), sum()사용
df.isnull().sum()          # ... 코드 입력

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [26]:
# 데이터 변경을 고려하여 df 를 df_1에 복사하기
# 원본인 df를 두고 변경하기 위해 deep copy 진행
df_1 = df.copy()
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(6), object(4)
memory usage: 83.7+ KB


In [27]:
# 결측치가 너무 많아서, Cabin 변수를 제거합니다.- drop() 사용
df_1 = df_1.drop(columns=["Cabin"])          # ... 코드 입력
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(6), object(3)
memory usage: 76.7+ KB


In [28]:
# 결측치 다시 한 번 확인하기
df_1.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [31]:
# 결측값이 존재하는 샘플 제거하고 null이 있는지 확인하고 그 합계 구하기 -isnull(),sum()
df_1.dropna(inplace=True)          # ... 코드 입력
df_1.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [32]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    int64  
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Embarked     712 non-null    object 
dtypes: float64(2), int64(6), object(3)
memory usage: 66.8+ KB


### 2)이상치 확인하기

In [33]:
# 'Age' 이상치 확인
df_1['Age']

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
885    39.0
886    27.0
887    19.0
889    26.0
890    32.0
Name: Age, Length: 712, dtype: float64

In [34]:
# (Age 값 - 내림 Age 값) 0 보다 크다면 소수점을 갖는 데이터로 분류합니다.
outlier = df_1[df_1['Age']-np.floor(df_1['Age']) > 0 ]['Age']

print('소수점을 갖는 Age 변수 이상치')
print(outlier)
print('이상치 처리 전 샘플 개수: %d' %(len(df_1)))
print('이상치 개수: %d' %(len(outlier)))

# 이상치를 처리합니다.
df_2 = df_1[df_1['Age']-np.floor(df_1['Age']) == 0 ]
print('이상치 처리 후 샘플 개수: %d' %(len(df_2)))
df_2 = np.floor(df_2['Age'])
df_2.head()

소수점을 갖는 Age 변수 이상치
57     28.50
78      0.83
111    14.50
116    70.50
122    32.50
123    32.50
148    36.50
152    55.50
153    40.50
203    45.50
227    20.50
296    23.50
305     0.92
331    45.50
469     0.75
525    40.50
644     0.75
676    24.50
735    28.50
755     0.67
767    30.50
803     0.42
814    30.50
831     0.83
843    34.50
Name: Age, dtype: float64
이상치 처리 전 샘플 개수: 712
이상치 개수: 25
이상치 처리 후 샘플 개수: 687


0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
Name: Age, dtype: float64

## 총 모음

In [36]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# 1.'titanic.csv' 데이터를 읽어옵니다.
df = pd.read_csv('./data/titanic.csv')
df.head()

# 1.데이터 전처리하기
# 1-1. 불필요하고 결측치가 많은 clolumns을 제거합니다.
df_1 = df.drop(columns=['Cabin','PassengerId','Name','Ticket'])

# 1-2.결측값이 존재하는 샘플 제거합니다.
df_2 = df_1.dropna()

# 1-3.이상치를 처리합니다.
df_3 = df_2[df_2['Age']-np.floor(df_2['Age']) == 0 ]

# 1-4.텍스트 데이터 숫자로 변환합니다.
# 1-4-1.replace()를 사용하여 male -> 0, female -> 1로 변환합니다.
df_4 = df_3.replace({'male':0, 'female':1})

# 1-4-2.get_dummies() 사용하여 원핫인코딩 처리하기
df_5 = pd.get_dummies(data = df_4, columns = ['Embarked'], prefix = 'Embarked')

# 1-5.데이터를 정규화합니다.
df_5['Fare'] =  normal(df_5['Fare'])
df_5['Age'] =  normal(df_5['Age'])

df_5

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,0.265823,1,0,0.014151,0,0,1
1,1,1,1,0.468354,1,0,0.139136,1,0,0
2,1,3,1,0.316456,0,0,0.015469,0,0,1
3,1,1,1,0.430380,1,0,0.103644,0,0,1
4,0,3,0,0.430380,0,0,0.015713,0,0,1
...,...,...,...,...,...,...,...,...,...,...
885,0,3,1,0.481013,0,5,0.056848,0,1,0
886,0,2,0,0.329114,0,0,0.025374,0,0,1
887,1,1,1,0.227848,0,0,0.058556,0,0,1
889,1,1,0,0.316456,0,0,0.058556,1,0,0


In [37]:
# index=False 주어야 기존 인덱스 값이 저장되지 않음
df_5.to_csv('./titanic_save.csv', index=False)